In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_Database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Zhigansk,RU,66.7697,123.3711,-1.71,100,87,8.25,overcast clouds
1,1,Katsuura,JP,35.1333,140.3000,55.78,63,99,9.82,overcast clouds
2,2,Lasa,IT,46.6166,10.7002,45.48,90,39,6.87,scattered clouds
3,3,Butaritari,KI,3.0707,172.7902,81.30,78,29,14.32,scattered clouds
4,4,Awbari,LY,26.5921,12.7805,68.29,29,0,15.99,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                254
City                   254
Country                253
Lat                    254
Lng                    254
Max Temp               254
Humidity               254
Cloudiness             254
Wind Speed             254
Current Description    254
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID                253
City                   253
Country                253
Lat                    253
Lng                    253
Max Temp               253
Humidity               253
Cloudiness             253
Wind Speed             253
Current Description    253
dtype: int64

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = None
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Butaritari,KI,81.30,scattered clouds,3.0707,172.7902,None
9,Kapaa,US,80.58,few clouds,22.0752,-159.3190,None
12,Mount Isa,AU,74.97,clear sky,-20.7333,139.5000,None
13,Rikitea,PF,78.12,light rain,-23.1203,-134.9692,None
16,Waingapu,ID,79.27,light rain,-9.6567,120.2641,None
24,Itarema,BR,79.09,broken clouds,-2.9248,-39.9167,None
26,Puerto Ayora,EC,80.56,broken clouds,-0.7393,-90.3518,None
30,Kariba,ZW,74.14,broken clouds,-16.5167,28.8000,None
33,Georgetown,MY,79.95,few clouds,5.4112,100.3354,None
35,Sao Jose Da Coroa Grande,BR,81.25,clear sky,-8.8978,-35.1478,None


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
hotel_df.head()

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Butaritari,KI,81.30,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
9,Kapaa,US,80.58,few clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
12,Mount Isa,AU,74.97,clear sky,-20.7333,139.5000,ibis Styles Mt Isa Verona
13,Rikitea,PF,78.12,light rain,-23.1203,-134.9692,People ThankYou
16,Waingapu,ID,79.27,light rain,-9.6567,120.2641,Pondok Wisata Elim


In [16]:
# Check for missing values.
hotel_df.count()

City                   253
Country                253
Max Temp               253
Current Description    253
Lat                    253
Lng                    253
Hotel Name             233
dtype: int64

In [18]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna(inplace=True)
hotel_df.count()

City                   233
Country                233
Max Temp               233
Current Description    233
Lat                    233
Lng                    233
Hotel Name             233
dtype: int64

In [19]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [24]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [25]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))